In [1]:
# 1. Import dependencies
import requests
import pymongo

import pandas as pd
import numpy
import requests
from bs4 import BeautifulSoup as bs

</br>
<center><font size=8, color="orange", face="Georgia"> Scrapped State Abbreviations</font><center>
<p>
<center><font size=4.5, color="black", face="Georgia"> Priamry key for joining tables</font><center>

![Abbreviations-Your Dictionary](./images/abbreviations.png)

# First Source -- STATES --

In [535]:
url = "https://abbreviations.yourdictionary.com/articles/state-abbrev.html"

In [536]:
#connect to url
response = requests.get(url)
response

<Response [200]>

In [537]:
#parse html page using Beautiful Soup
soup = bs(response.text, 'lxml')

# 50 USA States

In [538]:
#locate table containing all 50 state names and abbr. for 'div id'
states = soup.select('div[id^=article_body]')
for s in states:
    slt = soup.find('ul')
    #scrape names
    for a in slt:
        state_abbrev = slt.find_all('li')
        
# create new list to store items
state_list = []
#loop thorugh iterable list, abstract text value and appended each to new list
for one in state_abbrev:
    state_list.append(one.text) 

# Additional Territories

In [539]:
#locate list containing state and abbr.
abbrev = soup.find('div', class_='article_double')
#scrapped list
for a in abbrev:
    list_other = abbrev.find('ul')
    for a in list_other:
        others = list_other.find_all('li')
        
#loop thorugh iterable list, abstract text value and appended each to new list
for o in others:
    state_list.append(o.text) 

# Clean Data & Display DF

In [584]:
# save list as data frame
states = pd.DataFrame({'state':state_list})
# split columns to separate Abbrev and Name (will use it a primary key to join tables)
state = states['state'].str.split('-', 1, expand=True)
#rename col to 'State'
state_df = state.rename(columns={state.columns[0]: "State"}).set_index('State').sort_values(by='State', ascending=True)
state_df.count()
state_df

,1
State,
Alabama,AL
Alaska,AK
American Samoa,AS
Arizona,AZ
Arkansas,AR
California,CA
Colorado,CO
Connecticut,CT
Delaware,DE


<center><font size=8, color="green", face="Georgia"> Minimum Wage for all USA States from <br /> 1968 - 2018</font><center>

![Abbreviations-Your Dictionary](./images/deptlabor.png)

# Second Source -- WAGES --

In [546]:
#2007-2013
data4 = pd.read_html('https://www.dol.gov/whd/state/stateMinWageHis.htm',attrs={'summary': 'This is a table of minimum changes by state for 2007 to 2013.'})[0]
df4 = data4.reindex(data4.index.drop(0))
#print(df4)
#2014-2018
data5 = pd.read_html('https://www.dol.gov/whd/state/stateMinWageHis.htm',attrs={'summary': 'This is a table of minimum changes by state for 2014 to 2017.'})[0]
df5 = data5.reindex(data5.index.drop(0))
#df5

In [570]:
# 8. Merge all dfs on "State or other  jurisdiction". Note: Be mindful of spacing
# rename col for join, drop empty '...' values, TOTAL STATES :49, set index, sort descendig value
tests = df4.merge(df5,on='State or other  jurisdiction').rename(index=str, columns={"State or other  jurisdiction": "State"}).drop(['0', '17', '23', '39', '41']).set_index('State').sort_values(by='State', ascending=True)
tests.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
State,,,,,,,,,,,,
Alaska,7.15,7.15,7.15,7.75,7.75,7.75,7.75,7.75,8.75,9.75,9.80,9.84
Arizona,6.75,6.90,7.25,7.25,7.35,7.65,7.80,7.90,8.05,8.05,10.00,10.50
Arkansas,6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],7.50[c],8.00[c],8.50[c],8.50[c]
California,7.50,8.00,8.00,8.00,8.00,8.00,8.00,9.00,9.00,10.00,10.00,11.00
Colorado,6.85,7.02,7.28,7.24,7.36,7.64,7.78,8.00,8.23,8.31,9.30,10.20


# Merge: States & Wages

<center><font color="red" size=8>HELP</font></center>

In [590]:
#test 49   state_df 59


state_df.join(test, on='State', how='inner')

,1,Abbrev,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
State,,,,,,,,,,,,,,


# Obtained List from Repo
###  Manually cleansed list, limitation when merging results wouldnt display

In [575]:
states = ["AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", 
          "GU", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "PR", "RI", 
          "SD", "TX", "VI", "UT", "VT" ,"VA", "WA", "WV", "WI", "WY"]

tests.insert(0, 'Abbrev', states)

In [581]:
tests.head()

,Abbrev,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
State,,,,,,,,,,,,,
Alaska,AK,7.15,7.15,7.15,7.75,7.75,7.75,7.75,7.75,8.75,9.75,9.80,9.84
Arizona,AZ,6.75,6.90,7.25,7.25,7.35,7.65,7.80,7.90,8.05,8.05,10.00,10.50
Arkansas,AR,6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],6.25[c],7.50[c],8.00[c],8.50[c],8.50[c]
California,CA,7.50,8.00,8.00,8.00,8.00,8.00,8.00,9.00,9.00,10.00,10.00,11.00
Colorado,CO,6.85,7.02,7.28,7.24,7.36,7.64,7.78,8.00,8.23,8.31,9.30,10.20


# Below is the final output i want 

In [580]:
# 4. Display dataset saved from 'Source' (Source used:--KAGGLE: https://www.kaggle.com/shivamb/cities#zip_codes_states.csv)
df = pd.read_csv("./USA_cities.csv").rename(columns={'state': 'Abbrev'})


dfs_data = df.merge(tests, on="Abbrev")
dfs_data

,zip_code,latitude,longitude,city,Abbrev,county,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,501,40.922326,-72.637078,Holtsville,NY,Suffolk,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
1,544,40.922326,-72.637078,Holtsville,NY,Suffolk,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
2,6390,40.992288,-72.723496,Fishers Island,NY,Suffolk,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
3,10001,40.750422,-73.996328,New York,NY,New York,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
4,10002,40.718758,-73.986427,New York,NY,New York,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
5,10003,40.730223,-73.988564,New York,NY,New York,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
6,10004,40.696355,-74.025276,New York,NY,New York,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
7,10005,40.706903,-74.008654,New York,NY,New York,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
8,10006,40.708834,-74.013168,New York,NY,New York,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
9,10007,40.713941,-74.007401,New York,NY,New York,7.15,7.15,7.15,7.25,7.25,7.25,7.25,8.00,8.75,9.00,9.70,10.40
